In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9697,2021-05-30T17:00:00,ITA,19,Sicilia,38.115697,13.362357,484,65,549,9334,...,NaN,0.0,NaN,NaN,225551.0,0.0,2504820.0,1998521.0,ITG,ITG1
9698,2021-05-30T17:00:00,ITA,9,Toscana,43.769231,11.255889,422,110,532,7855,...,NaN,4.0,NaN,NaN,237872.0,3403.0,3659733.0,1067984.0,ITI,ITI1
9699,2021-05-30T17:00:00,ITA,10,Umbria,43.106758,12.388247,62,6,68,1490,...,"Si fa presente che 4 dei ricoveri NON UTI, non...",0.0,NaN,NaN,56377.0,0.0,924564.0,415843.0,ITI,ITI2
9700,2021-05-30T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,6,1,7,221,...,NaN,1.0,NaN,NaN,10934.0,649.0,96750.0,32716.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9697,2021-05-30T17:00:00,19,Sicilia,484,65,549,9334,9883,-105,348,...,225551,4503341,1722115.0,0.0,225551.0,0.0,2504820.0,1998521.0,ITG,ITG1
9698,2021-05-30T17:00:00,9,Toscana,422,110,532,7855,8387,-231,263,...,241275,4727717,2220142.0,4.0,237872.0,3403.0,3659733.0,1067984.0,ITI,ITI1
9699,2021-05-30T17:00:00,10,Umbria,62,6,68,1490,1558,-33,63,...,56377,1340407,377030.0,0.0,56377.0,0.0,924564.0,415843.0,ITI,ITI2
9700,2021-05-30T17:00:00,2,Valle d'Aosta,6,1,7,221,228,1,16,...,11583,129466,62674.0,1.0,10934.0,649.0,96750.0,32716.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-05-30', '2021-05-29')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-05-30T17:00:00,1,Piemonte,625,80,705,4674,5379,-206,233,...,360148,4519022,1846467.0,0.0,341660.0,18488.0,2908534.0,1610488.0,ITC,ITC1
1,2021-05-30T17:00:00,2,Valle d'Aosta,6,1,7,221,228,1,16,...,11583,129466,62674.0,1.0,10934.0,649.0,96750.0,32716.0,ITC,ITC2
2,2021-05-30T17:00:00,3,Lombardia,1076,246,1322,30741,32063,-312,458,...,834985,10666563,4254477.0,7.0,782202.0,52783.0,8885883.0,1780680.0,ITC,ITC4
3,2021-05-30T17:00:00,5,Veneto,288,49,337,8373,8710,-51,150,...,423267,7987725,1764951.0,1.0,411481.0,11786.0,5388937.0,2598788.0,ITH,ITH3
4,2021-05-30T17:00:00,6,Friuli Venezia Giulia,36,5,41,4875,4916,-17,34,...,106998,2024787,687127.0,0.0,92301.0,14697.0,1711564.0,313223.0,ITH,ITH4
5,2021-05-30T17:00:00,7,Liguria,121,31,152,1024,1176,-38,41,...,102718,1612531,641259.0,0.0,102718.0,0.0,1289914.0,322617.0,ITC,ITC3
6,2021-05-30T17:00:00,8,Emilia-Romagna,521,99,620,15142,15762,-301,190,...,383693,6243169,1862743.0,1.0,383378.0,315.0,4764737.0,1478432.0,ITH,ITH5
7,2021-05-30T17:00:00,9,Toscana,422,110,532,7855,8387,-231,263,...,241275,4727717,2220142.0,4.0,237872.0,3403.0,3659733.0,1067984.0,ITI,ITI1
8,2021-05-30T17:00:00,10,Umbria,62,6,68,1490,1558,-33,63,...,56377,1340407,377030.0,0.0,56377.0,0.0,924564.0,415843.0,ITI,ITI2
9,2021-05-30T17:00:00,11,Marche,113,28,141,3795,3936,-156,81,...,102632,1235590,731505.0,2.0,102632.0,0.0,1103064.0,132526.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-05-29T17:00:00,1,Piemonte,628,84,712,4873,5585,-288,149,...,359915,4508565,1842902.0,5.0,341434.0,18481.0,2905087.0,1603478.0,ITC,ITC1
1,2021-05-29T17:00:00,2,Valle d'Aosta,6,0,6,221,227,7,22,...,11567,129106,62531.0,0.0,10922.0,645.0,96571.0,32535.0,ITC,ITC2
2,2021-05-29T17:00:00,3,Lombardia,1131,243,1374,31001,32375,-145,620,...,834527,10638368,4242337.0,3.0,781819.0,52708.0,8867579.0,1770789.0,ITC,ITC4
3,2021-05-29T17:00:00,5,Veneto,292,50,342,8419,8761,-282,198,...,423117,7971908,1762848.0,1.0,411339.0,11778.0,5379780.0,2592128.0,ITH,ITH3
4,2021-05-29T17:00:00,6,Friuli Venezia Giulia,38,5,43,4890,4933,-19,76,...,106964,2021244,685906.0,0.0,92270.0,14694.0,1709389.0,311855.0,ITH,ITH4
5,2021-05-29T17:00:00,7,Liguria,117,33,150,1064,1214,-55,82,...,102677,1608785,639859.0,0.0,102677.0,0.0,1287634.0,321151.0,ITC,ITC3
6,2021-05-29T17:00:00,8,Emilia-Romagna,532,104,636,15427,16063,-327,224,...,383503,6231259,1861040.0,1.0,383188.0,315.0,4758248.0,1473011.0,ITH,ITH5
7,2021-05-29T17:00:00,9,Toscana,443,114,557,8061,8618,-513,200,...,241012,4711169,2213829.0,11.0,237620.0,3392.0,3649460.0,1061709.0,ITI,ITI1
8,2021-05-29T17:00:00,10,Umbria,68,6,74,1517,1591,-72,26,...,56314,1336248,376504.0,0.0,56314.0,0.0,922974.0,413274.0,ITI,ITI2
9,2021-05-29T17:00:00,11,Marche,120,30,150,3942,4092,-133,83,...,102551,1233360,729988.0,0.0,102551.0,0.0,1101257.0,132103.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-05-30T17:00:00,1,Piemonte,625,80,705,4674,5379,-206,233,...,4519022,1846467.0,0.0,341660.0,18488.0,2908534.0,1610488.0,ITC,ITC1,4.341375e+06
1,2021-05-30T17:00:00,2,Valle d'Aosta,6,1,7,221,228,1,16,...,129466,62674.0,1.0,10934.0,649.0,96750.0,32716.0,ITC,ITC2,1.255010e+05
2,2021-05-30T17:00:00,3,Lombardia,1076,246,1322,30741,32063,-312,458,...,10666563,4254477.0,7.0,782202.0,52783.0,8885883.0,1780680.0,ITC,ITC4,1.010397e+07
3,2021-05-30T17:00:00,5,Veneto,288,49,337,8373,8710,-51,150,...,7987725,1764951.0,1.0,411481.0,11786.0,5388937.0,2598788.0,ITH,ITH3,4.907704e+06
4,2021-05-30T17:00:00,6,Friuli Venezia Giulia,36,5,41,4875,4916,-17,34,...,2024787,687127.0,0.0,92301.0,14697.0,1711564.0,313223.0,ITH,ITH4,1.211357e+06
5,2021-05-30T17:00:00,7,Liguria,121,31,152,1024,1176,-38,41,...,1612531,641259.0,0.0,102718.0,0.0,1289914.0,322617.0,ITC,ITC3,1.543127e+06
6,2021-05-30T17:00:00,8,Emilia-Romagna,521,99,620,15142,15762,-301,190,...,6243169,1862743.0,1.0,383378.0,315.0,4764737.0,1478432.0,ITH,ITH5,4.467118e+06
7,2021-05-30T17:00:00,9,Toscana,422,110,532,7855,8387,-231,263,...,4727717,2220142.0,4.0,237872.0,3403.0,3659733.0,1067984.0,ITI,ITI1,3.722729e+06
8,2021-05-30T17:00:00,10,Umbria,62,6,68,1490,1558,-33,63,...,1340407,377030.0,0.0,56377.0,0.0,924564.0,415843.0,ITI,ITI2,8.802850e+05
9,2021-05-30T17:00:00,11,Marche,113,28,141,3795,3936,-156,81,...,1235590,731505.0,2.0,102632.0,0.0,1103064.0,132526.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-05-30T17:00:00,1,Piemonte,625,80,705,4674,5379,-206,233,...,1610488.0,ITC,ITC1,4.341375e+06,Piemonte,625,5824,80,628,99
1,2021-05-30T17:00:00,2,Valle d'Aosta,6,1,7,221,228,1,16,...,32716.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,6,227,1,30,6
2,2021-05-30T17:00:00,3,Lombardia,1076,246,1322,30741,32063,-312,458,...,1780680.0,ITC,ITC4,1.010397e+07,Lombardia,1076,8126,246,1416,114
3,2021-05-30T17:00:00,5,Veneto,288,49,337,8373,8710,-51,150,...,2598788.0,ITH,ITH3,4.907704e+06,Veneto,288,6000,49,1000,0
4,2021-05-30T17:00:00,6,Friuli Venezia Giulia,36,5,41,4875,4916,-17,34,...,313223.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,36,1277,5,175,0
5,2021-05-30T17:00:00,7,Liguria,121,31,152,1024,1176,-38,41,...,322617.0,ITC,ITC3,1.543127e+06,Liguria,121,1751,31,228,12
6,2021-05-30T17:00:00,8,Emilia-Romagna,521,99,620,15142,15762,-301,190,...,1478432.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,521,6538,99,760,0
7,2021-05-30T17:00:00,9,Toscana,422,110,532,7855,8387,-231,263,...,1067984.0,ITI,ITI1,3.722729e+06,Toscana,422,5033,110,518,24
8,2021-05-30T17:00:00,10,Umbria,62,6,68,1490,1558,-33,63,...,415843.0,ITI,ITI2,8.802850e+05,Umbria,62,687,6,116,0
9,2021-05-30T17:00:00,11,Marche,113,28,141,3795,3936,-156,81,...,132526.0,ITI,ITI3,1.518400e+06,Marche,113,1174,28,210,54


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-05-30T17:00:00,3,Lombardia,1076,246,1322,30741,32063,-312,458,...,17.37,-260,-162,28195,12140.0,5,765,1.62,0.00453,8.26
1,2021-05-30T17:00:00,15,Campania,733,61,794,66560,67354,-490,385,...,9.84,-472,54,13127,7124.0,1,874,2.93,0.00665,7.24
2,2021-05-30T17:00:00,19,Sicilia,484,65,549,9334,9883,-105,348,...,8.37,-80,-37,15841,4270.0,5,448,2.20,0.00700,4.54
3,2021-05-30T17:00:00,12,Lazio,913,152,1065,18750,19815,-981,278,...,16.12,-958,-29,20157,6531.0,6,1253,1.38,0.00474,5.83
4,2021-05-30T17:00:00,9,Toscana,422,110,532,7855,8387,-231,263,...,21.24,-206,63,16548,6313.0,8,486,1.59,0.00706,6.48
5,2021-05-30T17:00:00,1,Piemonte,625,80,705,4674,5379,-206,233,...,12.74,-199,84,10457,3565.0,1,438,2.23,0.00537,8.30
6,2021-05-30T17:00:00,8,Emilia-Romagna,521,99,620,15142,15762,-301,190,...,13.03,-285,-34,11910,1703.0,3,488,1.60,0.00425,8.59
7,2021-05-30T17:00:00,5,Veneto,288,49,337,8373,8710,-51,150,...,4.90,-46,-48,15817,2103.0,1,200,0.95,0.00306,8.62
8,2021-05-30T17:00:00,18,Calabria,243,17,260,9739,9999,-123,139,...,11.18,-115,-38,2212,1975.0,1,261,6.28,0.00722,3.47
9,2021-05-30T17:00:00,16,Puglia,572,63,635,25048,25683,-540,90,...,11.25,-504,-179,5395,468.0,4,626,1.67,0.00225,6.24


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Toscana,110,408,518,21.24
1,Lombardia,246,1170,1416,17.37
2,Lazio,152,791,943,16.12
3,P.A. Trento,13,77,90,14.44
4,Liguria,31,197,228,13.60
5,Marche,28,182,210,13.33
6,Emilia-Romagna,99,661,760,13.03
7,Piemonte,80,548,628,12.74
8,Puglia,63,497,560,11.25
9,Calabria,17,135,152,11.18


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))